In [ ]:
!pip install datasets sentence_transformers

In [ ]:
# Create your API token from your Hugging Face Account. Make sure to save it in text file or notepad for future use.
# Will need to add it once per section
from huggingface_hub import login
login()

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datasets import load_dataset
import time

class TextSimilarityModel:
    def __init__(self, corpus_name, rel_name, model_name='all-MiniLM-L6-v2', top_k=10):
        """
        Initialize the model with datasets and pre-trained sentence transformer.
        """
        self.model = SentenceTransformer(model_name)
        self.corpus_name = corpus_name
        self.rel_name = rel_name
        self.top_k = top_k
        self.load_data()


    def load_data(self):
        """
        Load and filter datasets based on test queries and documents.
        """
        # Load query and document datasets
        dataset_queries = load_dataset(self.corpus_name, "queries")
        dataset_docs = load_dataset(self.corpus_name, "corpus")

        # Extract queries and documents
        self.queries = dataset_queries["queries"]["text"]
        self.query_ids = dataset_queries["queries"]["_id"]
        self.documents = dataset_docs["corpus"]["text"]
        self.document_ids = dataset_docs["corpus"]["_id"]

                
        # Filter queries and documents and build relevant queries and documents mapping based on test set
        test_qrels = load_dataset(self.rel_name)["test"]
        self.filtered_test_query_ids = set(test_qrels["query-id"])
        self.filtered_test_doc_ids = set(test_qrels["corpus-id"])

        self.test_queries = [q for qid, q in zip(self.query_ids, self.queries) if qid in self.filtered_test_query_ids]
        self.test_query_ids = [qid for qid in self.query_ids if qid in self.filtered_test_query_ids]
        self.test_documents = [doc for did, doc in zip(self.document_ids, self.documents) if did in self.filtered_test_doc_ids]
        self.test_document_ids = [did for did in self.document_ids if did in self.filtered_test_doc_ids]

        self.test_query_id_to_relevant_doc_ids = {qid: [] for qid in self.test_query_ids}
        for qid, doc_id in zip(test_qrels["query-id"], test_qrels["corpus-id"]):
            if qid in self.test_query_id_to_relevant_doc_ids:
                self.test_query_id_to_relevant_doc_ids[qid].append(doc_id)
                
        ## Code Below this is used for creating the training set 
        # Build query and document id to text mapping
        self.query_id_to_text = {query_id:query for query_id, query in zip(self.query_ids, self.queries)}
        self.document_id_to_text = {document_id:document for document_id, document in zip(self.document_ids, self.documents)}

        # Build relevant queries and documents mapping based on train set
        train_qrels = load_dataset(self.rel_name)["train"]
        self.train_query_id_to_relevant_doc_ids = {qid: [] for qid in train_qrels["query-id"]}

        for qid, doc_id in zip(train_qrels["query-id"], train_qrels["corpus-id"]):
            if qid in self.train_query_id_to_relevant_doc_ids:
                # Append the document ID to the relevant doc mapping
                self.train_query_id_to_relevant_doc_ids[qid].append(doc_id)
        
        # Filter queries and documents and build relevant queries and documents mapping based on validation set  
        #TODO Put your code here. 
         ###########################################################################
       
        ###########################################################################
        

    #Task 1: Encode Queries and Documents (10 Pts)

    def encode_with_glove(self, glove_file_path: str, sentences: list[str]) -> list[np.ndarray]:

        """
        # Inputs:
            - glove_file_path (str): Path to the GloVe embeddings file (e.g., "glove.6B.50d.txt").
            - sentences (list[str]): A list of sentences to encode.

        # Output:
            - list[np.ndarray]: A list of sentence embeddings 
            
        (1) Encodes sentences by averaging GloVe 50d vectors of words in each sentence.
        (2) Return a sequence of embeddings of the sentences.
        Download the glove vectors from here. 
        https://nlp.stanford.edu/data/glove.6B.zip
        Handle unknown words by using zero vectors
        """
        #TODO Put your code here. 
        ###########################################################################
       
        ###########################################################################
    

    def encode_with_openai(
        sentences: List[str], 
        model: str = 'text-embedding-3-small',
        api_key: Optional[str] = None,
        batch_size: int = 100
    ) -> np.ndarray:
        """
        Encodes sentences using OpenAI's embedding API.
        
        # Inputs:
            - sentences (List[str]): A list of sentences to encode.
            - model (str): OpenAI model name. Options:
                * 'text-embedding-3-small' (1536 dims, $0.02/1M tokens) - RECOMMENDED
                * 'text-embedding-3-large' (3072 dims, $0.13/1M tokens)
                * 'text-embedding-ada-002' (1536 dims, legacy)
            - api_key (str, optional): OpenAI API key. If None, reads from OPENAI_API_KEY env variable
            - batch_size (int): Number of sentences to encode per API call (max 2048)
            
        Instructions:
        - Implement batched encoding with error handling
        - Add rate limiting (sleep between batches)
        
        Expected Cost for this Assignment:
        - ~4,000 texts (320 queries + 3,600 documents)
        - text-embedding-3-small: ~$0.08-0.10 per student
        - text-embedding-3-large: ~$0.50-0.65 per student
        
        Tips:
        - Use try-except for API errors
        - Implement retry logic with exponential backoff
        - Cache embeddings to avoid re-encoding
        - Monitor your usage at: https://platform.openai.com/usage
        """
        #TODO Put your code here.
        ###########################################################################
        
        ###########################################################################
        raise NotImplementedError("Task 1.5 (OpenAI) not implemented yet")

    #Task 2: Calculate Cosine Similarity and Rank Documents (20 Pts)
    
    def rank_documents(self, encoding_method: str = 'sentence_transformer') -> None:
        """
         # Inputs:
            - encoding_method (str): The method used for encoding queries/documents. 
                             Options: ['glove', 'sentence_transformer'].

        # Output:
            - None (updates self.query_id_to_ranked_doc_ids with ranked document IDs).
    
        (1) Compute cosine similarity between each document and the query
        (2) Rank documents for each query and save the results in a dictionary "query_id_to_ranked_doc_ids" 
            This will be used in "mean_average_precision"
            Example format {2: [125, 673], 35: [900, 822]}
        """
        if encoding_method == 'glove':
            # Note: Ensure "glove.6B.50d.txt" is downloaded and in the local directory
            query_embeddings = self.encode_with_glove("glove.6B.50d.txt", self.queries)
            document_embeddings = self.encode_with_glove("glove.6B.50d.txt", self.documents)
        elif encoding_method == 'sentence_transformer':
            query_embeddings = self.model.encode(self.queries)
            document_embeddings = self.model.encode(self.documents)
        elif encoding_method == 'openai':
            # Use environment variable or prompt for API key
            query_embeddings = self.encode_with_openai(self.queries)
            document_embeddings = self.encode_with_openai(self.documents)
        else:
            raise ValueError("Invalid encoding method. Choose 'glove' or 'sentence_transformer'.")
        
        
        #TODO Put your code here.
        ###########################################################################
         # define a dictionary to store the ranked documents for each query
        self.query_id_to_ranked_doc_ids = {}
      
        ###########################################################################

    @staticmethod
    def average_precision(relevant_docs: list[str], candidate_docs: list[str]) -> float:
        """
        # Inputs:
            - relevant_docs (list[str]): A list of document IDs that are relevant to the query.
            - candidate_docs (list[str]): A list of document IDs ranked by the model.

        # Output:
            - float: The average precision score
    
        Compute average precision for a single query.
        """
        y_true = [1 if doc_id in relevant_docs else 0 for doc_id in candidate_docs]
        precisions = [np.mean(y_true[:k+1]) for k in range(len(y_true)) if y_true[k]]
        return np.mean(precisions) if precisions else 0

    #Task 3: Calculate Evaluate System Performance (10 Pts)
    
    def mean_average_precision(self) -> float:
        """
        # Inputs:
            - None (uses ranked documents stored in self.query_id_to_ranked_doc_ids).

        # Output:
            - float: The MAP score, computed as the mean of all average precision scores.
    
        (1) Compute mean average precision for all queries using the "average_precision" function.
        (2) Compute the mean of all average precision scores
        Return the mean average precision score
        
        reference: https://www.evidentlyai.com/ranking-metrics/mean-average-precision-map
        https://towardsdatascience.com/map-mean-average-precision-might-confuse-you-5956f1bfa9e2
        """
         #TODO Put your code here. 
        ###########################################################################
        
        ###########################################################################
    
    #Task 4: Ranking the Top 10 Documents based on Similarity Scores (10 Pts)

    def show_ranking_documents(self, encoding_method: str, example_query: str) -> None:
                
        """
        # Inputs:
            - example_query (str): A query string for which top-ranked documents should be displayed.

        # Output:
            - None (prints the ranked documents along with similarity scores).
        
        (1) rank documents with given query with cosine similarity scores
        (2) prints the top 10 results along with its similarity score.
        
        """
        #TODO Put your code here. 

        ###########################################################################
      
        # 1. Encode the single query based on the method
        # 2. Reshape check: Ensure query_embedding is (1, n_features)
        # 3. Calculate scores
        
        
        ###########################################################################
      
    #Task 5:Fine tune the sentence transformer model (25 Pts)
    # Students are not graded on achieving a high MAP score. 
    # The key is to show understanding, experimentation, and thoughtful analysis.
    
    def fine_tune_model(self, batch_size: int = 32, num_epochs: int = 3, save_model_path: str = "finetuned_senBERT") -> None:

        """
        Fine-tunes the model using MultipleNegativesRankingLoss.
        (1) Prepare training examples from `self.prepare_training_examples()`
        (2) Experiment with [anchor, positive] vs [anchor, positive, negative]
        (3) Define a loss function (`MultipleNegativesRankingLoss`)
        (4) Freeze all model layers except the final layers
        (5) Train the model with the specified learning rate
        (6) Save the fine-tuned model
        """
        #TODO Put your code here.
        ###########################################################################

        ###########################################################################

    # Take a careful look into how the training set is created
    def prepare_training_examples(self) -> list[InputExample]:

        """
        Prepares training examples from the training data.
        # Inputs:
            - None (uses self.train_query_id_to_relevant_doc_ids to create training pairs).

         # Output:
            Output: - list[InputExample]: A list of training samples containing [anchor, positive] or [anchor, positive, negative].
            
        """
        train_examples = []
        for qid, doc_ids in self.train_query_id_to_relevant_doc_ids.items():
            for doc_id in doc_ids:
                anchor = self.query_id_to_text[qid]
                positive = self.document_id_to_text[doc_id]
                # TODO: Select random negative examples that are not relevant to the query.
                # TODO: Create list[InputExample] of type [anchor, positive, negative]
                
                train_examples.append(InputExample(texts=[anchor, positive]))

        return train_examples


In [ ]:
# Initialize the model with the medical dataset (nfcorpus)
model = TextSimilarityModel("BeIR/nfcorpus", "BeIR/nfcorpus-qrels")

# Evaluate using the default Sentence Transformer
print("Ranking with sentence_transformer...")
model.rank_documents(encoding_method='sentence_transformer')
sbert_map = model.mean_average_precision()
print("SBERT Mean Average Precision:", sbert_map)

# Evaluate using GloVe (requires 'glove.6B.50d.txt' in your directory)
print("\nRanking with glove...")
model.rank_documents(encoding_method='glove')
glove_map = model.mean_average_precision()
print("GloVe Mean Average Precision:", glove_map)

# Qualitative test: Show actual document text for a sample query
model.show_ranking_documents("glove","Breast Cancer Cells Feed on Cholesterol")
model.show_ranking_documents("sentence_transformer", "Breast Cancer Cells Feed on Cholesterol")

In [ ]:
print("\nRanking with openai...")
model.rank_documents(encoding_method='openai')
openai_map = model.mean_average_precision()
print("openai Mean Average Precision:", openai_map)

model.show_ranking_documents("openai","Breast Cancer Cells Feed on Cholesterol")

In [ ]:
# Finetune all-MiniLM-L6-v2 sentence transformer model
model.fine_tune_model(batch_size=3, num_epochs=2, save_model_path="finetuned_senBERT_train_v2")  # Adjust batch size and epochs as needed

model.rank_documents()
map_score = model.mean_average_precision()
print("Mean Average Precision:", map_score)